In [144]:
print("hello world")

hello world


In [145]:
path_to_train_images = './data/train/images/'
path_to_train_csv = ('./data/train/train.csv')

path_to_test_images = './data/test/images/'
path_to_test_csv = './data/test/test.csv'

In [146]:
#importing necessary packages
import keras
import os
import shutil
from os import listdir
from os.path import isfile, join
from keras.preprocessing.image import ImageDataGenerator
from keras import models
from keras.models import Model
from keras.models import Sequential
from keras.layers import Conv2D, Activation, MaxPooling2D, Dropout, GlobalAveragePooling1D, GlobalAveragePooling2D, Flatten, BatchNormalization, Dense


from keras.applications.inception_v3 import InceptionV3
from keras import optimizers
from keras.optimizers import *



In [147]:
train_images = [f for f in listdir(path_to_train_images) if isfile(join(path_to_train_images, f))]
test_images = [f for f in listdir(path_to_test_images) if isfile(join(path_to_test_images,f))]

In [148]:
#print(train_images[0])
print(test_images[0])

class1


In [149]:
import csv
def make_list_from_csv(path_to_csv):
    with open(path_to_csv,'r') as file:
        csv_reader = csv.reader(file)
        csv_contents = []
        for line in csv_reader:
            csv_contents.append(line)
        return csv_contents

train_csv_contents = make_list_from_csv(path_to_train_csv)
test_csv_contents = make_list_from_csv(path_to_test_csv)

In [150]:
print(train_csv_contents[0])
print(test_csv_contents[0])

['image_id', 'filename', 'label']
['image_id', 'filename']


In [151]:
print(train_csv_contents[1])
print(test_csv_contents[1])

['1', '1.jpg', '2']
['7201', '7201.jpg']


In [152]:
print(train_csv_contents[-1])
print(test_csv_contents[-1])

['7200', '7200.jpg', '2']
['12000', '12000.jpg']


In [158]:
#I want to group the images by the class they belong to. Then, I will write the images into subdirectories in the class they belong to. This will happen only in the training set. We do not know the classes in the test set.
def make_groupings_per_class(csv_contents,istestdir=False):
    insurance_claim_types = []
    distributions = {}
    csv_contents_sans_label = csv_contents[1:]
    for c in csv_contents_sans_label :
        #here, i want to make a small adjustment to the test set. The given test set does not have the ytest column that would help me to evaluate my model. instead, i have to rely on the predict function to blind-predict what classes will be without knowing the true labels. Therefore, it doesn't matter what class the test set belongs to, I will merge all of them into one directory just to create the image generator. 
        if istestdir:
            insur_type=1
        else:
            insur_type = int(c[-1])
        current_img_name = c[1]
        insurance_claim_types.append(insur_type)
        if insur_type in distributions.keys():
            existing_list = distributions[insur_type]
            existing_list.append(current_img_name)
        else:
            distributions[insur_type] = [current_img_name]
    return sorted(set(insurance_claim_types)), distributions
train_claim_classes, train_image_distributions = make_groupings_per_class(train_csv_contents)
test_claim_classes,test_image_distributions = make_groupings_per_class(test_csv_contents,istestdir=True)
"""
class 1- crack, 2 - scratch, 3 - flat tire, 4 - dent, 5 - glass shatter, 6 - lamp broken
"""
#print(train_claim_classes)
print(len(test_image_distributions[1]))

#CREATING SUBDIRECTORIES FOR EACH OF THE CLASSES
def refile_images_per_classes(image_dictionary,istestdir=False):
    for classname,list_of_images in image_dictionary.items():
        for l in list_of_images:
            try:
                if istestdir:
                    shutil.copy(path_to_test_images+l, './data/test/images/class'+str(classname))
                else:
                    shutil.copy(path_to_train_images+l, './data/train/images/class'+str(classname))
                os.remove(path_to_test_images+l)
                os.remove(path_to_train_images+l)
            except FileNotFoundError:
                print('already moved the file, please check respective class sub directory')
        
#refile_images_per_classes(train_image_distributions)
#refile_images_per_classes(test_image_distributions,istestdir=True)

4800


In [154]:
#preparing for preprocessing and transormation for training
rescale = 1./255
target_size = (100,100)
batch_size = 100
class_mode = 'categorical'

train_data_generator = ImageDataGenerator(rescale=rescale,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)
train_generator = train_data_generator.flow_from_directory(
    path_to_train_images,
    target_size=target_size,
    class_mode=class_mode,
    batch_size=batch_size,
    shuffle=True)




Found 7200 images belonging to 6 classes.
